# Selecting spectral lines

In [ ]:
import pandas as pd
import spectrograph as sg
import matplotlib
import matplotlib.pyplot as plt
from matplotlib import cm
#from colorspacious import cspace_converter
import numpy as np
import warnings
warnings.filterwarnings('ignore')

In [ ]:
%matplotlib widget
%load_ext autoreload
%autoreload 2

In [ ]:
data = sg.CCD2d(**sg.default_kwargs)
cmap = cm.get_cmap('rainbow')
colors = cmap(np.linspace(0,1, data.get_orders().shape[0]))
orders = data.get_orders()

We now write all order as a single function $x = f(\lambda o)$

In [ ]:
plt.figure(figsize=(5,4))
for o in orders:
    I = data.index_order(o)
    plt.plot(data.o[I]*data.l[I], data.x[I], 'o', color=colors[o-orders[0]])
plt.show()

Removing a first global polynomial of low order..

In [ ]:
ol = data.o * data.l
global_fit = np.polynomial.Polynomial.fit(sg.link_ol(ol), data.x, deg=4)
# residuals
res_1 = data.x - global_fit(sg.link_ol(ol))
# plotting the residuum
plt.figure(figsize=(4,3))
for o in orders:
    I = data.index_order(o)
    plt.plot(ol[I], res_1[I], '.', color=colors[o-orders[0]])
plt.show()

reducing by linear $o$ dependency
$$
x = P_1(ol)+P_2(o)
$$
Degree of $P_1$ is $4$, degree of $P_2$ is $2$.

In [ ]:
fit_polynomial2 = np.polynomial.Polynomial.fit(data.o, res_1, deg=2)
res_2 = res_1 - fit_polynomial2(data.o)
# plotting the residuum
plt.figure(figsize=(4,3))
for o in orders:
    I = data.index_order(o)
    plt.plot(ol[I], res_2[I], '.', color=colors[o-orders[0]])
    
# single out the strange order....(plotted in cyan)
o = 54
I = data.index_order(o)
plt.plot(ol[I], res_2[I], '.', color='k')
plt.show()

In [ ]:
# new set of orders
orders = np.array([23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39,
       40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 55, 56,
       57, 58])
plt.figure(figsize=(4,3))
for o in orders:
    I = data.index_order(o)
    plt.plot(ol[I], res_2[I], '.', color=colors[o-orders[0]])

refitting all with order 54 removed

In [ ]:
global_fit = np.polynomial.Polynomial.fit(sg.link_ol(ol), data.x, deg=4)
res_1 = data.x - global_fit(sg.link_ol(ol))
fit_polynomial2 = np.polynomial.Polynomial.fit(data.o, res_1, deg=3)
res_2 = res_1 - fit_polynomial2(data.o)
fit_polynomial3 = np.polynomial.Polynomial.fit(sg.link_ol(ol)*data.o, res_2, deg=2)
res_3 = res_2 - fit_polynomial3(sg.link_ol(ol))
plt.figure(figsize=(4,3))
for o in orders:
    I = data.index_order(o)
    plt.plot(ol[I], res_3[I], '.', color=colors[o-orders[0]])

In [ ]:
o = 57   # play with me....
I = data.index_order(o)
poly = np.polynomial.Polynomial.fit(sg.link_ol(ol[I]), res_3[I], deg = 6)
res_4 = res_3[I] - poly(sg.link_ol(ol[I]))

sigma = data.sigma
plt.figure(figsize=(5,4))
plt.plot(ol[I], res_4, '.', color=colors[o-orders[0]])
plt.show()

Analysing residuum around polynomial for each order

In [ ]:
n =  6 # order of polynomial
bins_abs = np.linspace(-1, 1, 21)
bins_rel = np.linspace(-5, 5, 21)
for o in orders:
    I = data.index_order(o)
    sig = np.sqrt(0.01 + data.sigma**2)
    p = np.polynomial.Polynomial.fit(sg.link_ol(ol[I]), data.x[I], n, w=1./sig[I])
    res = (data.x[I] - p(sg.link_ol(ol[I])))
    #pd.cut(res, np.linspace(-4,4,5))
    #res.hist(bins=10)
    plt.figure(figsize=(5,2))
    plt.subplot(131)
    plt.title('abs '+str(o))
    plt.hist(res, bins=bins_abs, range=(-0.5, 0.5))
    plt.subplot(132)
    plt.title('rel ' + str(o))
    plt.hist(res/sig[I], bins=bins_rel, range=(-5,5))
    plt.subplot(133)
    plt.plot(data.l[I], res, '.', color=colors[o-orders[0]])
plt.show()

### Outlier removal order by order
criteria: absolut $\epsilon < \sigma_a$ and relative $\epsilon/\sigma < \sigma_r$ 
We convert $\sigma_a$ in errors of radial velocity using the formula
$$
\delta_{rv} = \gamma_o \,\sigma_a,\quad \gamma_o = 3\cdot 10^8 \frac{d\lambda/dx}{\lambda} = \frac{3\cdot 10^{8} }{\lambda |dx/d\lambda|_o}=\frac{3\cdot 10^{8} \cdot o }{\lambda |\partial_{o\lambda}x(o\lambda, o)|}
$$
where $x=x(o\lambda, o)$ is taken in linear global approximation

In [ ]:
sigma_a = 0.5
sigma_r = 3
# global linear approximation
p_lin = np.polynomial.Polynomial.fit(ol, data.x, deg=1)
gamma_o = {o: o * 3e8 / (data.l[data.index_order(o)].mean() * p_lin.coef[1]) for o in orders}
gamma = o * 3e8 / (data.l * p_lin.coef[1])
plt.figure(figsize=(5,4))
for o in orders:
    plt.plot( [o], [gamma_o[o]], 'o', color=colors[o-orders[0]])
plt.xlabel('order')
plt.ylabel('average $\gamma$ factor')
plt.show()

Therefore with a target resolution of $100 m/s$ the reuired precision depending on the order needs to be adjusted 

In [ ]:
delta_vr = np.linspace(0, 200, 50)
Io = np.any([data.index_order(o) for o in orders], axis=0)
plt.figure(figsize=(5,4))
plt.plot(delta_vr, [ sum(gamma[Io] * data.sigma[Io] < d_vr) for d_vr in delta_vr])
plt.title('theoretical precision per data')
plt.xlabel('target rv precision')
plt.ylabel('number of usable spectral lines')
plt.show()

In [ ]:
delta_vr = np.linspace(0, 200, 50)
Io = np.any([data.index_order(o) for o in orders], axis=0)
plt.figure(figsize=(5,4))
for o in orders:
    I = data.index_order(o)
    plt.plot(delta_vr, [ sum(gamma[I] * data.sigma[I] < d_vr) for d_vr in delta_vr], color=colors[o-orders[0]])
plt.title('theoretical precision per data order by order')
plt.xlabel('target rv precision')
plt.ylabel('number of usable spectral lines')
plt.show()

### 2D modeling

Data is $x_n = g(o_n\lambda, o_n) + f_{o_n}(o_n\lambda_n) + \epsilon_n$ with orders $o_n \in [22, 58]$.
We take the global shape model $g$ of the form
$$
g(o\lambda,o) = \sum_{k=0}^{N_k}\sum_{i=0}^{N_i} C_{kmi} \tilde T_i(o) \tilde T_k(o\lambda)
$$
with degree $N_k=2$ and $N_i=1$. This covers the space of full 2s polynomials of this low degree. This part has a Bayesian prior which is flat so this this part adapts freely
$$
\mathbb{P}(C) \sim 1
$$
The individual fluctuations $f_o$ for each order are higeher degree polynomials.
$$
f_o = \sum_{m=0}^{N_r} D_{o, m} \tilde T_m 
$$
Here smootheness constraints are added.
First we penalise the total size of fluctuations
$$
\sum_o \Vert f_o\Vert^2 = \sum |D_{o,m}|^2
$$
The difference between sucessive orders is controlled by
$$
\sum_o \Vert f_{o+1}-f_o \Vert^2 = \sum |D_{o+1,m}|^2 + |D_{o,m}|^2 - 2D_{o+1,m}D_{o,m}
$$
Here we have used the orthogonality of the Chebushev polynomials. 
To controll higher order smoothness in $o$ direction we also conisder the discrete Laplacian
$$
\sum_o \Vert f_{o+2}-2f_{o+1} + f_o \Vert^2 =
\sum |D_{o+2, m}|^2 + 4|D_{o+1, m}|^2 + |D_{o,m}|^2 - 2 D_{o+2,m}D_{o+1,m} - 2 D_{o+1, m} D_{o,m} + D_{o+2,m}D_{o,m}
$$
These terms are quadratic forms, that can be cast into a matrix equation as
$$
\sum_o \Vert f_o\Vert^2 = D^T \mathbb{I} D,\quad \sum_o \Vert f_{o+1}-f_o \Vert^2 = D^T\Gamma D,\quad
\sum_o \Vert f_{o+2}-2f_{o+1} + f_o \Vert^2 = D^T \Xi D
$$

In [ ]:
Ni = 8 # orders of o
Nk = 7 # orders of ol

Nr = 10 # degree in olambda direction

olrange = (ol.min(), ol.max())
orange = (orders.min(), orders.max())
I = np.identity(Nr+1)
# Computing all Tchebyshev polynomials we need in ol
Tsheb = [np.polynomial.chebyshev.Chebyshev(c, window=[-1,1], domain=olrange) for c in I]

I = np.identity(Ni+1)
Tshebo = [np.polynomial.chebyshev.Chebyshev(c, window=[-1,1], domain=orange) for c in I]
ndata = len(data.data) # number of data
G = np.zeros((ndata, (Nk+1)*(Ni+1)))

ik = [(i, k) for i in range(Ni+1) for k in range(Nk+1)] # this fixes the order of basis

for u, olol in enumerate(zip(ol, data.o)):
    ooll, o = olol
    for v, j in enumerate(ik):
        i, k = j
        G[u, v] = Tshebo[i](o)*Tsheb[k](ooll)

s = 1./np.sqrt((1/gamma)**2 + data.sigma**2)

def get_Gsme(I):
    s = 1./np.sqrt((1/gamma)**2 + data.sigma**2)
    return s[I][:, np.newaxis] * G[I,:]

def fit_free_model(I):
    smeG = get_Gsme(I)
    C = np.linalg.lstsq(smeG, s[I] * data.x[I])[0]
    return C, G



## sigma clipping based on RV limit on each spectral line

In [ ]:
target_rv = 100
I = Io
Iold = I
change = True

rv = 10000

while change:
    C, G = fit_free_model(I)
    dx = data.x - np.dot(G,C)
    I = gamma * dx <= rv
    I = I & Io
    change = (not np.all( Iold == I)) | rv>target_rv
    Iold = I
    rv = int(0.2*(target_rv+1 + 4*rv)-0.1)

Isigma_clipped = I
print('remaining number of points', sum(I), rv)

C, G = fit_free_model(Isigma_clipped)

res_5 = data.x[Io] - np.dot(G,C)[Io]
res_5
plt.figure(figsize=(3,1))
plt.hist(gamma[Io]*res_5, bins=np.linspace(-1000,1000,30))
plt.show()

def g(ol, o):
    return sum([C[l]*Tshebo[ii](o) * Tsheb[kk](ol) for l, (ii, kk) in enumerate(ik)])

def ggo(o):
    """
    Chebysheb series at given o
    """
    return sum([C[l]*Tshebo[ii](o) * Tsheb[kk] for l, (ii, kk) in enumerate(ik)])

def ig(x, o):
    """
    input:
    :x: pixel position
    :o: order
    returns: 
    :ol: order X lambda
    """
    # first order proxy
    lloo = (C[0]*Tsheb[0]+C[1]*Tsheb[1]-x).roots()[0]
    # polynome to solve
    gg = ggo(o) - x
    rts = gg.roots() # all roots
    I = np.argmin(np.abs(rts-lloo))  # getting zero closest to lloo
    return np.real(rts[I])

res = data.x - g(ol, data.o)
plt.figure(figsize=(4,3))
for o in orders:
    I = data.index_order(o) & Isigma_clipped
    plt.plot(ol[I], res[I], '.', color=colors[o-orders[0]])

In [ ]:
# p = np.polynomial.Polynomial.fit(ol[Isigma_clipped], data.x[Isigma_clipped], deg=4) 
# for plotting
# alternatively the firs
p = lambda ol: g(ol, min(orders))
res = data.x[Isigma_clipped] - p(ol[Isigma_clipped])

plt.figure(figsize=(4,3))
ols = np.linspace(ol.min(), ol.max(), 1024)
for o in set(data.o):
    I = data.index_order(o) & Isigma_clipped
    ols = np.linspace(ol.min(), ol.max(), 1024)
    plt.plot(ols, g(ols, o) - p(ols), '-.', color='k', linewidth=0.5)
    ols = np.linspace(ol[I].min(), ol[I].max(), 1024)
    plt.plot(ols, g(ols, o) - p(ols), '-', color=colors[o-orders[0]])
    plt.plot(ol[I], res[I], '.', color=colors[o-orders[0]])
plt.ylim(0,175)
plt.show()


In [ ]:
x = np.arange(74)
oo = np.arange(data.o.min(), data.o.max()+1)
ordermap = [ ig(xx,o)/o for o in oo for xx in x ]

In [ ]:
xx = np.arange(len(ordermap))
plt.figure(figsize=(3,2))
plt.plot(np.mod(xx,74), ordermap, '.')
plt.show()

# Two dimensional modeling

In [ ]:
ik

In [ ]:
(300+sum(np.array([1,2,-3]) * Tsheb[:3])).roots()

In [ ]:
sum(np.array([1,2,-3]) * Tsheb[:3])(np.array([195696.38775984, 257292.42171349]))

In [ ]:
Gs = get_Gsme(Isigma_clipped)
GtG = np.dot(Gs.T,Gs)

def var_ol_o(ol, o):
    """
    posterior variance / estimation variance at ol, o
    """
    xx = [Tshebo[ii](o)*Tsheb[kk](ol) for (ii,kk) in ik ]
    return np.dot(xx, np.linalg.solve(GtG,xx))

np.sqrt(var_ol_o(222000, 22))